# Exercises & extra-curriculum

#### 1. Turn the code to get the data (from section 1. Get Data above) into a Python script, such as get_data.py.
* When you run the script using python get_data.py it should check if the data already exists and skip downloading if it does.
* If the data download is successful, you should be able to access the pizza_steak_sushi images from the data directory.

In [17]:
%%writefile going_modular/get_data.py 
import os 
import zipfile 

from pathlib import Path

import requests

# Create directory if needed
data_path = Path(".data")
if not data_path.is_dir():
    print(f"Creating the '{str(data_path)}' directory...")
    data_path.mkdir(parents=True)
else:
    print(f"Directory '{str(data_path)}' already exists, skipping this step.")

# Download the data if needed 
if not any(data_path.iterdir()): 
    with open(data_path/"pizza_steak_sushi.zip", "wb") as f:
        print(f"Downloading the zip data file...")
        request = requests.get("https://github.com/mrdbourke/pytorch-deep-learning/raw/main/data/pizza_steak_sushi.zip")
        f.write(request.content)

    # Unzipping data
    image_path = data_path/"pizza_steak_sushi"
    with zipfile.ZipFile(data_path/"pizza_steak_sushi.zip") as zip_f:
        print("Unzipping data...")
        zip_f.extractall(image_path)
    print(f"Removing zipfile...")
    os.remove(data_path/"pizza_steak_sushi.zip")
else:
    print(f"Data already downloaded, skipping this step.")

Overwriting going_modular/get_data.py


#### 2. Use Python's argparse module to be able to send the train.py custom hyperparameter values for training procedures.
1. Add an argument for using a different:
    * Training/testing directory
    * Learning rate
    * Batch size
    * Number of epochs to train for
    * Number of hidden units in the TinyVGG model
2. Keep the default values for each of the above arguments as what they already are (as in notebook 05).
3. For example, you should be able to run something similar to the following line to train a TinyVGG model with a learning rate of 0.003 and a batch size of 64 for 20 epochs: python train.py --learning_rate 0.003 --batch_size 64 --num_epochs 20.
4. Note: Since train.py leverages the other scripts we created in section 05, such as, model_builder.py, utils.py and engine.py, you'll have to make sure they're available to use too. You can find these in the going_modular folder on the course GitHub.

In [44]:
!python going_modular/train.py 

Train data:
Dataset ImageFolder
    Number of datapoints: 225
    Root location: .data/pizza_steak_sushi/train
    StandardTransform
Transform: Compose(
               Resize(size=(64, 64), interpolation=bilinear, max_size=None, antialias=None)
               ToTensor()
           )
Test data:
Dataset ImageFolder
    Number of datapoints: 75
    Root location: .data/pizza_steak_sushi/test
    StandardTransform
Transform: Compose(
               Resize(size=(64, 64), interpolation=bilinear, max_size=None, antialias=None)
               ToTensor()
           )
--------------------- Hyperparameters ---------------------
Data path:       .data          | Batch size:      32             
Learning rate:   0.001          | Number of epochs:5              
Hidden units:    10             
-----------------------------------------------------------
-------------- Epoch 0 --------------
Train metrics: (1.0981, 0.30)
Test metrics: (1.0961, 0.27)
-------------- Epoch 1 --------------
Train metrics

In [48]:
%%writefile going_modular/train.py

'''
Trains a PyTorch image classification model using device-agnostic code.
''' 

import os 
import torch
from torchvision import transforms
import data_setup, engine, model_builder, utils
from timeit import default_timer as timer
from utils import fit_text
import argparse

# Getting arguments
parser = argparse.ArgumentParser()


data_group= parser.add_argument_group("Data")
data_group.add_argument("--data_path", type=str, default=".data", metavar="DIR",
                    help="Path where data is stored. (type: %(type)s, default: %(default)s)")
data_group.add_argument("--batch_size", type=int, default=32, metavar="BATCH",
                    help="Size of each batch in the dataloder. (type: %(type)s, default: %(default)s)")

train_group = parser.add_argument_group("Training")
train_group.add_argument("--epochs", type=int, default=5, metavar="EPOCHS",
                    help="Number of epochs to train the model. (type: %(type)s, default: %(default)s)")
train_group.add_argument("--learning_rate", type=float, default=0.001, metavar="LR",
                    help="Optimizer's learning rate. (type: %(type)s, default: %(default)s)")

model_group = parser.add_argument_group("Model")
model_group.add_argument("--hidden_units", type=int, default=10, metavar="H_UNITS",
                    help="Number of hidden units in the neural network. (type: %(type)s, default: %(default)s)")

options = parser.parse_args()

torch.manual_seed(42)
torch.cuda.manual_seed(42)

# Setup hyperparameters
NUM_EPOCHS = options.epochs
BATCH_SIZE = options.batch_size
HIDDEN_UNITS = options.hidden_units
LEARNING_RATE = options.learning_rate
DATA_PATH = options.data_path

# Setup directories 
train_dir = DATA_PATH+"/pizza_steak_sushi/train"
test_dir = DATA_PATH+"/pizza_steak_sushi/test"

# Device agnostic-code
device = "cuda" if torch.cuda.is_available() else "cpu"

# Create transforms
data_transform = transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.ToTensor()
])

# Dataloders & class names 
train_dataloader, test_dataloader, class_names = data_setup.create_dataloaders(train_dir=train_dir,
                                                                  test_dir=test_dir,
                                                                  transform=data_transform,
                                                                  batch_size=BATCH_SIZE)
# Create the model
model = model_builder.TinyVGG(in_c=3,
                              out_shape=len(class_names),
                              hidden_units=HIDDEN_UNITS).to(device)

# Setup loss & optimizer
loss_fn = torch.nn.CrossEntropyLoss()
optim = torch.optim.Adam(params=model.parameters(),
                        lr=LEARNING_RATE)

# Print out hyperparameters
print("--------------------- Hyperparameters ---------------------")
print(f"{'Data path:':<17}{fit_text(DATA_PATH,15)}| {'Batch size:':<17}{fit_text(BATCH_SIZE,15)}")
print(f"{'Learning rate:':<17}{fit_text(LEARNING_RATE,15)}| {'Number of epochs:':<17}{fit_text(NUM_EPOCHS,15)}")
print(f"{'Hidden units:':<17}{fit_text(HIDDEN_UNITS,15)}")
print("-----------------------------------------------------------")


# Train the model
torch.manual_seed(42)
torch.cuda.manual_seed(42)

start_t = timer()
results = engine.train(model=model,
                       loss_fn=loss_fn,
                       optimizer=optim,
                       train_dataloader=train_dataloader,
                       test_dataloader=test_dataloader,
                       epochs=NUM_EPOCHS,
                       dev=device)
end_t = timer()
print(f"[INFO]: Training time: {end_t-start_t:.3f}")

# Save the model
utils.save_model(model=model,
                 target_dir="models",
                 model_name="0.5_going_modular_script_mode_TinyVGG.pth")



Overwriting going_modular/train.py


#### 3. Create a script to predict (such as predict.py) on a target image given a file path with a saved model.
* For example, you should be able to run the command python predict.py some_image.jpeg and have a trained PyTorch model predict on the image and return its prediction.
* To see example prediction code, check out the predicting on a custom image section in notebook 04.
* You may also have to write code to load in a trained model.

In [2]:
# Downloading the image
from pathlib import Path
import requests as rq

data_path = Path(".data")
if not data_path.exists():
    print(f"Creating data directory as: '{data_path}'...")
    data_path.mkdir()
else:
    print(f"Directory already exists, skipping this step.")
    
custom_image_path = data_path/"custom_image.jpg"
if not custom_image_path.exists():
    print(f"Downloading image at: '{custom_image_path}'...")
    url = "https://imgs.search.brave.com/3Ul7nnUJ3C532AlLy9D_RXR98XL4CRDzLN0gFf_Y-x8/rs:fit:860:0:0:0/g:ce/aHR0cHM6Ly9saDUu/Z29vZ2xldXNlcmNv/bnRlbnQuY29tL3Av/QUYxUWlwTkpmRUFx/a3hnVGZqektTZUwt/ekRGc0Q2VFBKWVYz/bjA0VVRCeUk9dzgw/MC1oNTAwLWstbm8"
    request = rq.get(url=url)
    with open(custom_image_path, "wb") as f:
        f.write(request.content)
else:
    print(f"Image already downladed, skipping this step.")
    

Directory already exists, skipping this step.


In [70]:
%%writefile going_modular/predict.py 
from torchvision.io import read_image
from torchvision import transforms
from pathlib import Path
import argparse 
import torch
from model_builder import TinyVGG
from utils import fit_text

dev = "cuda" if torch.cuda.is_available() else "cpu"


parser = argparse.ArgumentParser()

parser.add_argument("image_path", metavar="IMG", type=str,
                    help="Path to the image for which the prediction is desired.")
parser.add_argument("--model_path", metavar="MODEL", type=str ,
                    default="./models/0.5_going_modular_script_mode_TinyVGG.pth",
                    help="Path to the model that will make the inference.")
parser.add_argument("--hidden_units", metavar="UNITS", default=10, type=int,
                    help="Number of hidden units used in the model.")
parser.add_argument("image_class", metavar="CLASS", type=str,
                    help="Image class, it must be one between: [pizza, steak, sushi].")

options = parser.parse_args()
HIDDEN_UNITS = options.hidden_units
MODEL_PATH = options.model_path
IMAGE_PATH = options.image_path
IMAGE_CLASS = options.image_class

# Print out options
print("--------------------- Hyperparameters ---------------------")
print(f"{'Image path:':<17}{fit_text(IMAGE_PATH,15)}| {'Model path:':<17}{fit_text(MODEL_PATH,15)}")
print(f"{'Hidden units:':<17}{fit_text(HIDDEN_UNITS,15)}| {'Class:':<17}{fit_text(IMAGE_CLASS,15)}")
print("-----------------------------------------------------------")


################# Getting the image #################model_builder i
transform = transforms.Compose([
    transforms.Resize((64, 64)),
])
image = transform(read_image(IMAGE_PATH).type(dtype=torch.float32).to(dev)/255)

################# Getting the model #################
loaded__model = TinyVGG(in_c=3,
                        out_shape=3,
                        hidden_units=HIDDEN_UNITS)
state_dict = torch.load(MODEL_PATH)
loaded__model.load_state_dict(state_dict)

################# Making the prediction #################
class_names = ["pizza", "steak", "sushi"]
real_label = [IMAGE_CLASS]
print(f"Image shape: {image.shape}")
with torch.inference_mode():
    loaded__model.eval()
    logits = loaded__model(image.unsqueeze(dim=0))
    pred = torch.argmax(logits, dim=1).item()

################# Printing out #################
print(f"Real class: {IMAGE_CLASS} | Prediction: {class_names[pred]}")

Overwriting going_modular/predict.py


In [71]:
!python going_modular/predict.py --hidden_units 20 .data/custom_image.jpg sushi

--------------------- Hyperparameters ---------------------
Image path:      .data/custom...| Model path:      ./models/0.5...
Hidden units:    20             | Class:           sushi          
-----------------------------------------------------------
Image shape: torch.Size([3, 64, 64])
Real class: sushi | Prediction: sushi


In [55]:
!python going_modular/predict.py -h

usage: predict.py [-h] [--model_path MODEL] [--hidden_units UNITS] IMG CLASS

positional arguments:
  IMG                   Path to the image for which the prediction is desired.
  CLASS                 Image class, it must be one between: [pizza, steak,
                        sushi].

optional arguments:
  -h, --help            show this help message and exit
  --model_path MODEL    Path to the model that will make the inference.
  --hidden_units UNITS  Number of hidden units used in the model.
